In [1]:
import pybullet as p
import pybullet_data
import time
import numpy as np
import cvxpy as cp
import threading
import random
import tkinter as tk
from tkinter import messagebox
from tkinter import Tk, Label, IntVar, Scale, HORIZONTAL
from tkinter import Tk, Canvas, Label

pybullet build time: Nov 28 2023 23:45:17


In [2]:
import xml.etree.ElementTree as ET

def parse_collision_boxes(urdf_file):
    # Parse the URDF file
    tree = ET.parse(urdf_file)
    root = tree.getroot()

    boxes = []
    
    for link in root.findall("link"):
        for collision in link.findall("collision"):
            geometry = collision.find("geometry")
            box = geometry.find("box")
            if box is not None:
                size = list(map(float, box.attrib["size"].split()))
                origin_elem = collision.find("origin")
                if origin_elem is not None:
                    origin = list(map(float, origin_elem.attrib["xyz"].split()))
                    rpy = list(map(float, origin_elem.attrib["rpy"].split()))
                else:
                    origin = [0.0, 0.0, 0.0]
                    rpy = [0.0, 0.0, 0.0]
                boxes.append((size, origin, rpy))
    return boxes

def compute_min_max(size, origin, rpy):
    # Unpack size and origin
    sx, sy, sz = size
    ox, oy, oz = origin
    
    # Half extents
    hx, hy, hz = sx / 2, sy / 2, sz / 2
    
    # Define corners of the box relative to origin
    corners = np.array([
        [-hx, -hy, -hz],
        [-hx, -hy, hz],
        [-hx, hy, -hz],
        [-hx, hy, hz],
        [hx, -hy, -hz],
        [hx, -hy, hz],
        [hx, hy, -hz],
        [hx, hy, hz],
    ])
    
    # Rotation matrix for z-axis only (rpy[2])
    theta = rpy[2]
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta),  np.cos(theta), 0],
        [0, 0, 1]
    ])
    
    # Rotate and translate corners
    transformed_corners = np.dot(corners, rotation_matrix.T) + np.array([ox, oy, oz])
    
    # Get min and max
    min_coords = transformed_corners.min(axis=0)
    max_coords = transformed_corners.max(axis=0)
    
    return min_coords, max_coords

In [3]:
urdf_file = "model.urdf"
boxes = parse_collision_boxes(urdf_file)

global_min = np.array([float('inf'), float('inf'), float('inf')])
global_max = np.array([-float('inf'), -float('inf'), -float('inf')])

mins = []
maxs = []
for size, origin, rpy in boxes:
    min_coords, max_coords = compute_min_max(size, origin, rpy)
    mins.append(min_coords)
    maxs.append(max_coords)
    
    # Update global bounds
    global_min = np.minimum(global_min, min_coords)
    global_max = np.maximum(global_max, max_coords)

print("\nGlobal Bounds:")
print(f"  Global Min: {global_min.round(3)}")
print(f"  Global Max: {global_max.round(3)}")


Global Bounds:
  Global Min: [-2.275 -2.525  0.   ]
  Global Max: [2.275 2.025 0.3  ]


In [4]:
buffer=0.1

In [5]:
bounds = np.vstack((global_min, global_max)).T
adjusted_min = np.array(mins) - buffer
adjusted_max = np.array(maxs) + buffer
bounds[2, 0] += buffer
bounds[2, 1] -= buffer

In [6]:
def is_point_in_cuboid(point, min_corner, max_corner):
    """
    Check if a point is inside an axis-aligned cuboid.
    """
    return all(min_corner[i] <= point[i] <= max_corner[i] for i in range(3))

def does_line_segment_intersect_cuboid(p1, p2, min_corner, max_corner):
    """
    Efficiently checks if a line segment intersects an axis-aligned cuboid (AABB).
    """
    d = p2 - p1  # Direction vector of the line segment
    t_min, t_max = 0, 1  # Parametric range for the segment

    for axis in range(3):  # Iterate over x, y, z axes
        if d[axis] != 0:  # The segment is not parallel to this axis
            t1 = (min_corner[axis] - p1[axis]) / d[axis]
            t2 = (max_corner[axis] - p1[axis]) / d[axis]

            t_near = min(t1, t2)
            t_far = max(t1, t2)

            t_min = max(t_min, t_near)
            t_max = min(t_max, t_far)

            if t_min > t_max:  # The segment exits before entering
                return False
        else:  # The segment is parallel to this axis
            if p1[axis] < min_corner[axis] or p1[axis] > max_corner[axis]:
                return False  # The segment is outside the slab on this axis

    return True  # The segment intersects the cuboid

In [7]:
def check_collision(point):
    for i in range(len(boxes)):
        if is_point_in_cuboid(point, adjusted_min[i], adjusted_max[i]):
            return True
    return False

In [8]:
def check_collision_line(point1, point2):
    for i in range(len(boxes)):
        if does_line_segment_intersect_cuboid(point1, point2, adjusted_min[i], adjusted_max[i]):
            return True
    return False

In [9]:
def update_sphere_color(sphere_id, sphere_position):
    # Condition: Change color if the sphere's x position is greater than 1
    if check_collision(sphere_position):
        # Set color to red
        p.changeVisualShape(sphere_id, -1, rgbaColor=[1, 0, 0, 1])
    else:
        # Set color to green
        p.changeVisualShape(sphere_id, -1, rgbaColor=[0, 1, 0, 1])

In [10]:
def check_line(sphere_id, droneId):
    robot_position, robot_orientation = p.getBasePositionAndOrientation(droneId)
    robot_position = np.array(robot_position)
    sphere_position, sphere_orientation = p.getBasePositionAndOrientation(sphere_id)
    sphere_position = np.array(sphere_position)    
    if check_collision_line(robot_position, sphere_position):
        # Set color to red
        p.changeVisualShape(droneId, -1, rgbaColor=[1, 0, 0, 1])
    else:
        # Set color to green
        p.changeVisualShape(droneId, -1, rgbaColor=[0, 1, 0, 1])

In [11]:
def euclidean_distance(point1, point2):
    point1 = np.array(point1)  # Convert to numpy array if not already
    point2 = np.array(point2)  # Convert to numpy array if not already
    return np.linalg.norm(point1 - point2)

In [12]:
def RRT_build(q_0, q_goal, n, bounds):
    V = [q_0]  # List of vertices (nodes)
    E = []  # List of edges (connections between nodes)
    global goal_reached
    goal_reached = False
    count = 0

    collision_free_goal = not check_collision_line(q_0, q_goal)

    # If the path to the goal is also collision-free, add the goal and edge, then stop
    if collision_free_goal:
        #print('path to goal good')
        E.append([q_0, q_goal])
        V.append(q_goal)
        goal_reached = True
        # show_success_popup()
    
    for _ in range(n):
        count = count + 1
        #print('New sample:', count)
        if goal_reached:  # If the goal is reached, exit the loop
            break
        
        q = np.array([random.uniform(bounds[0, 0], bounds[0, 1]), random.uniform(bounds[1, 0], bounds[1, 1]), random.uniform(bounds[2, 0], bounds[2, 1])])

        # Keep sampling new points if they collide with an obstacle
        while check_collision(q):
            q = np.array([random.uniform(bounds[0, 0], bounds[0, 1]), random.uniform(bounds[1, 0], bounds[1, 1]), random.uniform(bounds[2, 0], bounds[2, 1])])
        
        # Find the closest existing node in the tree
        distances = [np.linalg.norm(q - v) for v in V]
        closest_neighbour = V[np.argmin(distances)]

        # Interpolate between the new point and the closest neighbour
        collision_free = not check_collision_line(q, closest_neighbour)

        # If the path is collision-free, add the point and edge to the tree
        if collision_free:
            V.append(q)
            E.append([q, closest_neighbour])
            #print('collision free')

            # Check if the path to the goal is collision-free
            collision_free_goal = not check_collision_line(q, q_goal)

            # If the path to the goal is also collision-free, add the goal and edge, then stop
            if collision_free_goal:
                #print('path to goal good')
                E.append([q, q_goal])
                V.append(q_goal)
                goal_reached = True
                # show_success_popup()
                # break
    return V, E

# Function to reconstruct the path from the tree
def find_RRT_path(E, q_0, q_goal):
    global goal_reached
    if goal_reached == False:
        path = [q_0]
        print("No path is found within given iterations")
        show_warning_popup()
    else:
        path = [q_goal, E[-1][0]]  # Start path with the goal
        while not np.allclose(path[-1], q_0, atol=1e-6):  # Loop until the start is reached
            for edge in E:
                if np.allclose(edge[0], path[-1], atol=1e-6):  # Find the edge leading to the last point in the path
                    path.append(edge[1])  # Add the next point in the path
                    break
        path.reverse()  # Reverse the path to go from start to goal
    return path


def rrt_final(q_0, q_goal, n, bounds):
    #print("Initial:", q_0, "Goal: ", q_goal)
    V, E = RRT_build(q_0, q_goal, n, bounds)
    #print("V: ", V, "E: ", E)
    path = find_RRT_path(E, q_0, q_goal)
    #print(path)
    return path


def show_warning_popup():
    # Initialize Tkinter
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    # Show a warning message box
    messagebox.showwarning("Warning", "Not enough iterations!")


# def show_success_popup():
#     # Initialize Tkinter
#     root = tk.Tk()
#     root.withdraw()  # Hide the main window

#     # Show a warning message box
#     messagebox.showwarning("Yay!", "You reached the goal!")


In [13]:
threshold = 0.01

In [14]:
def follower(robot_id, target_position, speed, bounds):
    robot_position, robot_orientation = p.getBasePositionAndOrientation(robot_id)
    robot_position = np.array(robot_position)
    target_position = np.array(target_position)
    path = rrt_final(robot_position, target_position, 5000, bounds)
    for node in path:
        target_position = node
        while euclidean_distance(robot_position[:2], target_position[:2]) > threshold:
            difference = target_position - robot_position
            #difference[2] = 0
            robot_position = robot_position + (speed/ np.linalg.norm(difference))*difference
            p.resetBasePositionAndOrientation(robot_id, list(robot_position), robot_orientation)
            p.stepSimulation()
            time.sleep(1. / 240.)

In [15]:
sphere_ids = []

In [16]:
def update_sphere_positions(positions):
    # Access the global sphere_ids
    global sphere_ids
    # For example: move each sphere randomly every time
    for i in range(positions.shape[1]):
        new_position = positions[:, i]
        p.resetBasePositionAndOrientation(sphere_ids[i], list(new_position), [0, 0, 0, 1])  # Update position

In [17]:
def get_initial_sphere_positions(num_spheres=60):
    # Example function to generate initial positions for all spheres
    positions = []
    for _ in range(num_spheres):
        x = 0
        y = 0
        z = 0
        positions.append([x, y, z])
    return positions

In [18]:
def mpc_control(N, x_init, v_init, x_target, dt, buffer):
    weight_tracking = 1.0*np.eye(3)
    cost = 0.
    constraints = []
    m = 3.0  # mass of the drone
    g = 9.81  # gravitational acceleration
    gravity_acceleration = np.array([0, 0, -g])
    max_thrust_z = 80.0  # Maximum thrust in z (vertical) direction in N
    max_thrust_xy = 40.0  # Maximum thrust in x and y directions in N
    max_total_thrust = 80.0  # Maximum total thrust in N (sum of x, y, z)
    k_drag_per_m = np.array([0.5, 0.5, 0.8]) / m
    
    # Create the optimization variables
    x = cp.Variable((3, N + 1))
    v = cp.Variable((3, N + 1))
    u = cp.Variable((3, N))

    # Direction of the cylinder (axis)
    d = x_target - x_init
    d_norm = np.linalg.norm(d)
    d_unit = d / d_norm

    for k in range(N):
        # Tracking cost
        cost += cp.quad_form(x[:, k] - x_target, weight_tracking)
        
        # State dynamics constraints
        constraints += [x[:, k+1] == x[:, k] + dt * v[:, k]]
        constraints += [v[:, k+1] == v[:, k] + dt * (u[:, k] / m + gravity_acceleration - cp.multiply(k_drag_per_m, v[:, k]))]
        
        # Thrust constraints
        constraints += [cp.abs(u[0, k]) <= max_thrust_xy]
        constraints += [cp.abs(u[1, k]) <= max_thrust_xy]
        constraints += [cp.abs(u[2, k]) <= max_thrust_z]
        constraints += [cp.norm(u[:, k], 2) <= max_total_thrust]

        # Cylinder radial constraint
        # Projection of x onto the axis of the cylinder
        projection = x_init + cp.multiply((x[:, k] - x_init).T @ d_unit, d_unit)
        radial_distance = cp.norm(x[:, k] - projection, 2)
        constraints += [radial_distance <= buffer]

        # Extended cylinder length constraint (allow up to r distance before x_init and after x_target)
        projection_length = (x[:, k] - x_init).T @ d_unit
        # constraints += [projection_length >= -buffer]  # Allow up to r distance before x_init
        constraints += [projection_length <= d_norm + buffer]  # Allow up to r distance beyond x_target

    # Initial conditions
    constraints += [x[:, 0] == x_init]
    constraints += [v[:, 0] == v_init]
    
    # Solves the problem
    problem = cp.Problem(cp.Minimize(cost), constraints)
    problem.solve(solver=cp.CLARABEL)

    # start_scs = time.time()
    # problem.solve(solver=cp.SCS)
    # time_scs = time.time() - start_scs

    # start_ecos = time.time()
    # problem.solve(solver=cp.ECOS)
    # time_ecos = time.time() - start_ecos
    
    # start_clarabel = time.time()
    # problem.solve(solver=cp.CLARABEL)
    # time_clarabel = time.time() - start_clarabel
    
    # print(f"SCS Time: {time_scs:.6f}s")
    # print(f"ECOS Time: {time_ecos:.6f}s")
    # print(f"CLARABEL Time: {time_clarabel:.6f}s")
    return x[:, 1:].value, v[:, 1:].value, u.value

In [19]:
import numpy as np

def find_closest_state(current_robot_state, predicted_states):
    """
    Finds the index of the state in predicted_states that is closest to current_robot_state.
    
    Args:
        current_robot_state (np.ndarray): The robot's current state (e.g., [x, y, z]).
        predicted_states (np.ndarray): The predicted states (e.g., shape (3, N)).
    
    Returns:
        int: The index of the closest state in predicted_states.
    """
    # Calculate the Euclidean distance between the current state and each predicted state
    distances = np.linalg.norm(predicted_states.T - current_robot_state, axis=1)
    # Return the index of the minimum distance
    return np.argmin(distances)


In [20]:
global_mpc_value = np.zeros(3)

In [21]:
def draw_or_remove_lines(points, draw=True, line_color=[0, 0, 0], line_width=5.0):
    """
    Draws or removes lines connecting all the points in the provided numpy array in the PyBullet environment.

    Args:
        points (np.ndarray): A numpy array of shape (N, 3) containing the coordinates of the points.
        draw (bool): If True, draw the lines. If False, remove all previously drawn lines.
        line_color (list): A list of 3 or 4 elements representing the RGB(A) color of the line.
        line_width (float): The width of the line (only applicable when drawing lines).
    Returns:
        list: A list of IDs for the drawn lines (if `draw=True`).
    """
    if not isinstance(points, np.ndarray):
        raise ValueError("The 'points' parameter must be a numpy array.")
    if points.shape[1] != 3:
        raise ValueError("Each point in 'points' must have exactly 3 coordinates.")
    if len(points) < 2:
        raise ValueError("At least two points are required to draw a line.")

    # Store debug line IDs
    line_ids = []

    if draw:
        for i in range(len(points) - 1):
            start_point = points[i]
            end_point = points[i + 1]
            line_id = p.addUserDebugLine(
                start_point, end_point, lineColorRGB=line_color[:3], lineWidth=line_width
            )
            line_ids.append(line_id)
        return line_ids
    else:
        # Remove all debug items (or you can use a list of IDs if tracked)
        p.removeAllUserDebugItems()
        return None


In [22]:
def follower_mpc(robot_id, target_position, bounds, buffer):
    global global_mpc_value
    robot_position, robot_orientation = p.getBasePositionAndOrientation(robot_id)
    robot_position = np.array(robot_position)
    target_position = np.array(target_position)
    path = rrt_final(robot_position, target_position, 5000, bounds)
    draw_or_remove_lines(np.array(path))
    velocity = np.zeros(3)
    threshold_mpc = 0.02
    N = 15
    freq = 15
    used_states_limit = 45
    dt = 1 / freq
    m = 3.0  # mass of the drone
    g = 9.81  # gravitational acceleration
    gravity_force = np.array([0, 0, -m * g])
    k_drag = np.array([0.5, 0.5, 0.8])
    
    i = 0
    while i < len(path):
        while (i + 1) < len(path) and not check_collision_line(robot_position, path[i + 1]):
            i += 1
        target_position = path[i]
        while euclidean_distance(robot_position, target_position) > threshold_mpc:
            while (i + 1) < len(path) and not check_collision_line(robot_position, path[i + 1]):
                i += 1
            target_position = path[i]
            next_states, next_velocities, thrusts = mpc_control(N, robot_position, velocity, target_position, dt, 3*buffer)
            update_sphere_positions(next_states)
            next_state = next_states[:, 0]
            next_velocity = next_velocities[:, 0]
            thrust = thrusts[:, 0]                
            global_mpc_value = thrust
            robot_position = next_state
            velocity = next_velocity
            p.resetBasePositionAndOrientation(robot_id, list(robot_position), robot_orientation)
            p.stepSimulation()

        i += 1

    global_mpc_value = np.zeros(3)
    draw_or_remove_lines(np.array(path), False)

In [23]:
def run_simulation():
# Connect to PyBullet with GUI
    global global_mpc_value
    global sphere_ids
    physicsClient = p.connect(p.GUI)

    p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
    
    # Load environment
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.8)
    p.loadURDF("plane.urdf")  # Ground plane
    
    urdf_path = "model.urdf"
    robot_id = p.loadURDF(urdf_path, basePosition=[0, 0, 0.5])  # Adjust the position as needed
    
    # Set starting position and orientation for the drone
    #startPos = [0, -0.5, 2]
    startPos = [0, -0.5, 0.1]
    startOrientation = p.getQuaternionFromEuler([0, 0, 0])
    # Load a sample drone (use a valid URDF file here)
    # This scales the robot by adjusting the position for scaling (approximates rescaling)
    #droneId = p.loadURDF("r2d2.urdf", startPos, startOrientation, globalScaling=0.5)
    
    sphere_radius = 0.05
    sphere_visual_shape = p.GEOM_SPHERE
    sphere_color = [1, 1, 1, 1]  # White color

    sphere_positions = get_initial_sphere_positions()  # Get positions for spheres

    for position in sphere_positions:
        sphere_visual = p.createVisualShape(sphere_visual_shape, radius=sphere_radius, rgbaColor=sphere_color)
        sphere_idx = p.createMultiBody(baseMass=0,  # Static object
                                      baseVisualShapeIndex=sphere_visual,
                                      basePosition=position)
        sphere_ids.append(sphere_idx)


    
    # Add a sphere to the simulation
    sphere_radius = 0.1
    sphere_visual_shape = p.GEOM_SPHERE
    #sphere_collision_shape = p.createCollisionShape(sphere_visual_shape, radius=sphere_radius)
    sphere_visual = p.createVisualShape(sphere_visual_shape, radius=sphere_radius, rgbaColor=[1, 0, 0, 1])
    
    sphere_start_position = [0, 0, 0.1]
    # sphere_id = p.createMultiBody(baseMass=0,  # Static object
    #                               baseCollisionShapeIndex=sphere_collision_shape,
    #                               baseVisualShapeIndex=sphere_visual,
    #                               basePosition=sphere_start_position)
    sphere_id = p.createMultiBody(baseMass=0,  # Static object
                                  baseVisualShapeIndex=sphere_visual,
                                  basePosition=sphere_start_position)
    droneId = p.createMultiBody(baseMass=0,  # Static object
                                  baseVisualShapeIndex=sphere_visual,
                                  basePosition=startPos)
    # Adjust the camera to focus on the drone
    p.resetDebugVisualizerCamera(cameraDistance=3, cameraYaw=50, cameraPitch=-30, cameraTargetPosition=startPos)
    
    # Sphere movement variables
    sphere_speed = 0.01
    sphere_position = list(sphere_start_position)
    
    # Run simulation
    while True:
        keys = p.getKeyboardEvents()
        if ord('q') in keys and keys[ord('q')] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
            break
        # Move sphere continuously based on key input
        if p.B3G_UP_ARROW in keys and keys[p.B3G_UP_ARROW] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
            sphere_position[1] += sphere_speed
        if p.B3G_DOWN_ARROW in keys and keys[p.B3G_DOWN_ARROW] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
            sphere_position[1] -= sphere_speed
        if p.B3G_LEFT_ARROW in keys and keys[p.B3G_LEFT_ARROW] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
            sphere_position[0] -= sphere_speed
        if p.B3G_RIGHT_ARROW in keys and keys[p.B3G_RIGHT_ARROW] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
            sphere_position[0] += sphere_speed
        if ord('f') in keys and keys[ord('f')] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
            p.changeVisualShape(sphere_id, -1, rgbaColor=[0, 0, 1, 1])    
            follower(droneId, sphere_position, sphere_speed, bounds)
    
        if ord('m') in keys and keys[ord('m')] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
            p.changeVisualShape(sphere_id, -1, rgbaColor=[0, 0, 1, 1])    
            follower_mpc(droneId, sphere_position, bounds, buffer)
            
        if ord('c') in keys and keys[ord('c')] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
            p.changeVisualShape(sphere_id, -1, rgbaColor=[0, 0, 1, 1])    
            check_line(sphere_id, droneId)
        
        
        # Update sphere position
        p.resetBasePositionAndOrientation(sphere_id, sphere_position, [0, 0, 0, 1])
    
        update_sphere_color(sphere_id, np.array(sphere_position))
    
        # Step simulation
        p.stepSimulation()
        time.sleep(1. / 240.)  # Slow simulation for visualization
    
    # Disconnect
    p.disconnect()

In [24]:
import numpy as np
from tkinter import *
import matplotlib.pyplot as plt

def create_progress_gui():
    global global_mpc_value  # Access the global variable (np.array of shape (3,))

    root = Tk()
    root.title("Drone Thrust")  # Window title

    # X Thrust (MPC Value 0)
    label_x = Label(root, text="X thrust (-40 to 40)", font=("Arial", 14))
    label_x.pack(pady=10)

    canvas_width = 400  # Width of the canvas
    canvas_height = 40  # Height of the canvas
    canvas_x = Canvas(root, width=canvas_width, height=canvas_height, bg="black", highlightthickness=0)
    canvas_x.pack(pady=20)

    # Y Thrust (MPC Value 1)
    label_y = Label(root, text="Y thrust (-40 to 40)", font=("Arial", 14))
    label_y.pack(pady=10)

    canvas_y = Canvas(root, width=40, height=400, bg="black", highlightthickness=0)  # Height increased
    canvas_y.pack(pady=20)

    # Z Thrust (MPC Value 2, Range -80 to 80)
    label_z = Label(root, text="Z thrust (-80 to 80)", font=("Arial", 14))
    label_z.pack(pady=10)

    canvas_z = Canvas(root, width=400, height=40, bg="black", highlightthickness=0)  # Width increased
    canvas_z.pack(pady=20)

    # Display values for smooth transitions
    display_values = [0, 0, 0]  # Mutable list for smoothing all three bars

    # Function to always return red color
    def get_color(value, value_range):
        # Return red color (255, 0, 0)
        return '#ff0000'  # Hex representation of red color

    # Function to update all three bars dynamically
    def update_bars():
        nonlocal display_values  # Access the mutable display values
        target_values = np.array(global_mpc_value)  # Ensure target_values is a NumPy array

        # Smoothly adjust the display values towards the targets
        smoothing_factor = 0.2  # Adjust the rate of change (lower is smoother)
        for i in range(3):
            display_values[i] += (target_values[i] - display_values[i]) * smoothing_factor

        # Clear the canvases
        canvas_x.delete("all")
        canvas_y.delete("all")
        canvas_z.delete("all")

        # Update X Thrust Bar (MPC Value 0)
        rect_center_x = (display_values[0] + 40) / 80 * canvas_width
        rect_width_x = 60  # Doubled the width of the colored region
        rect_left_x = max(0, rect_center_x - rect_width_x / 2)
        rect_right_x = min(canvas_width, rect_center_x + rect_width_x / 2)
        color_x = get_color(display_values[0], (-40, 40))
        canvas_x.create_rectangle(rect_left_x, 0, rect_right_x, canvas_height, fill=color_x, outline="")

        # Update Y Thrust Bar (MPC Value 1)
        rect_center_y = (40 - display_values[1]) / 80 * canvas_y.winfo_height()
        rect_height_y = 60  # Doubled the height of the colored region
        rect_top_y = max(0, rect_center_y - rect_height_y / 2)
        rect_bottom_y = min(canvas_y.winfo_height(), rect_center_y + rect_height_y / 2)
        color_y = get_color(display_values[1], (-40, 40))
        canvas_y.create_rectangle(0, rect_top_y, canvas_y.winfo_width(), rect_bottom_y, fill=color_y, outline="")

        # Update Z Thrust Bar (MPC Value 2, Range -80 to 80)
        rect_center_z = (display_values[2] + 80) / 160 * canvas_width
        rect_width_z = 60  # Doubled the width of the colored region
        rect_left_z = max(0, rect_center_z - rect_width_z / 2)
        rect_right_z = min(canvas_width, rect_center_z + rect_width_z / 2)
        color_z = get_color(display_values[2], (-80, 80))
        canvas_z.create_rectangle(rect_left_z, 0, rect_right_z, canvas_height, fill=color_z, outline="")

        # Schedule the next update
        root.after(50, update_bars)  # Update every 50 ms

    update_bars()  # Start the periodic update
    root.mainloop()

In [25]:
simulation_thread = threading.Thread(target=run_simulation)
simulation_thread.daemon = True
simulation_thread.start()

create_progress_gui()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 550.120
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 550.120
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial dat